In [27]:
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from utils import aggregate_by_ticker, get_last_closing_price, get_full_price_history

## Import

In [28]:
io_path = Path('..','data','in')

In [29]:
df_storico = pd.read_excel(
    io_path / Path('demo.xlsx'),
    sheet_name='Storico',
    dtype={
        'Borsa': str,
        'Ticker': str,
        'Quote': int,
        'Prezzo (€)': float,
        'Commissioni': float,
    }
).rename(
    columns={
        'Borsa': 'exchange',
        'Ticker': 'ticker',
        'Data Operazione': 'transaction_date',
        'Quote': 'shares',
        'Prezzo (€)': 'price',
        'Commissioni (€)': 'fees',
    }
)

In [30]:
df_anagrafica = pd.read_excel(
    io_path / Path('demo.xlsx'),
    sheet_name='Anagrafica Titoli',
    dtype=str
).rename(
    columns={
        'Ticker': 'ticker',
        'Nome ETF': 'name',
        'Tipologia': 'asset_class',
        'Macro Tipologia': 'macro_asset_class',
    }
)

In [31]:
df_pf = aggregate_by_ticker(df_storico, in_pf_only=True)

## Ultima chiusura

In [32]:
ticker_list = df_pf['ticker_yf'].to_list()

df_last_closing = get_last_closing_price(ticker_list=ticker_list)

## PMC *vs* prezzo attuale

In [33]:
df_j = df_pf[['ticker_yf','dca','shares']].merge(
    df_last_closing[['ticker_yf','price']],
    how='left',
    on='ticker_yf'
)

df_j['gain'] = np.where(
    df_j['price'].gt(df_j['dca']),
    True,
    False,
)

## PnL

In [34]:
expense = (
    df_j['shares'] * df_j['dca']
).sum()

In [35]:
fees = df_storico['fees'].sum().round(2)

In [36]:
pf_actual_value = (
    df_j['shares'] * df_j['price']
).sum()

In [37]:
(pf_actual_value - expense).round(2), (pf_actual_value - expense - fees).round(2)

(91.89, 73.15)

In [38]:
np.round(
    100 * (pf_actual_value - expense) / expense,
    1
), np.round(
    100 * (pf_actual_value - expense - fees) / expense,
    1
)

(2.2, 1.7)

## Pivot per tipologia

In [39]:
df_j['ticker'] = df_j['ticker_yf'].str.split('.').str[0]
df_j['position_value'] = df_j['shares'] * df_j['price']

In [40]:
df_pivot = df_j.merge(
    df_anagrafica,
    how='left',
    on='ticker'
).groupby(
    [
        'macro_asset_class',
        'asset_class',
        'ticker_yf',
        'name',
    ]
)['position_value'].sum().reset_index()

In [41]:
df_pivot['weight_pf'] = (
    100 * df_pivot['position_value'].div(pf_actual_value)
).astype(float).round(1)

In [42]:
pd.pivot_table(
    df_pivot,
    values=['weight_pf'],
    index=['macro_asset_class', 'asset_class'],
    aggfunc='sum',
    margins=True,
    margins_name='Total',
)

weight_pf
macro_asset_class asset_class                        
Azionario         Azionario Emergente             4.4
                  Azionario Sviluppato           63.4
Cash USD          Obbligazionario ST US           2.5
Commodities       Oro                             4.0
Obbligazionario   Obbligazionario IL EU           3.7
                  Obbligazionario LT EU          14.4
                  Obbligazionario Mondiale        2.7
                  Obbligazionario ST EU           5.0
Total                                           100.1

In [43]:
df_cool = df_j.merge(
    df_anagrafica,
    how='left',
    on='ticker'
)

df_cool['pnl'] = (
    (df_cool['price'] - df_cool['dca']) * df_cool['shares']
).astype(float).round(1)

In [44]:
df_pnl = df_cool.groupby(
    ['macro_asset_class','asset_class']
)['pnl'].sum().reset_index().sort_values(['macro_asset_class','asset_class'])

## Full History

In [45]:
df_full_history = get_full_price_history(ticker_list)

In [46]:
df_full_history_concat = pd.concat(
    [df_full_history[t_] for t_ in ticker_list],
    axis=1,
)

In [47]:
# First not-null row
first_idx = df_full_history_concat.apply(
    pd.Series.first_valid_index
).max()

df = df_full_history_concat.loc[first_idx:]

print(f'Starting from {str(first_idx)[:10]} ({df.shape[0]} days, {round(df.shape[0]/252, 1)} yrs)')

Starting from 2019-07-02 (1034 days, 4.1 yrs)


In [49]:
df.tail(20)

,LCWD.MI,AGGH.MI,36BZ.DE,ICGA.DE,EM710.MI,FLXI.DE,MWRD.MI,CSBGE3.MI,EMI.MI,IB01.L,SGLD.MI
2023-06-12,13.922,4.6010,4.1885,3.9995,156.960007,31.025000,95.919998,107.180000,161.350006,105.110001,175.770004
2023-06-13,14.052,4.5970,4.1920,4.0265,156.460007,31.320000,96.900002,107.099998,161.100006,105.110001,174.009995
2023-06-14,14.072,4.5980,4.2055,4.0415,156.190002,31.219999,96.919998,107.059998,161.009995,105.150002,174.270004
2023-06-15,14.032,4.6005,4.2400,4.0905,155.529999,31.014999,96.610001,106.870003,161.100006,105.199997,172.839996
2023-06-16,14.090,4.6030,4.2555,4.0965,156.229996,31.190001,97.000000,106.910004,162.130005,105.180000,173.149994
2023-06-19,14.006,4.5900,4.2275,4.0790,155.440002,31.080000,96.360001,106.839996,161.490005,105.220001,172.550003
2023-06-20,13.948,4.6090,4.1855,3.9505,156.660004,31.174999,95.930000,106.900002,162.699997,105.230003,171.289993
2023-06-21,13.876,4.6035,4.1180,3.8940,156.330002,31.084999,95.470001,106.870003,162.610001,105.239998,170.270004
2023-06-22,13.848,4.5965,4.1025,3.8810,155.490005,30.975000,95.180000,106.739998,161.889999,105.300003,168.619995
2023-06-23,13.806,4.6100,4.0765,3.8195,157.039993,30.760000,95.110001,106.919998,163.270004,105.279999,170.500000


## Grafichetti

[tipo](https://plotly.com/python/horizontal-bar-charts/)

In [ ]:
fig = px.sunburst(
    df_pivot.assign(hole=" "),
    path=['hole','macro_asset_class','asset_class','ticker_yf'],
    values='position_value',
)

fig.show()

## Sharpe, Sortino, Drawdon

[link](https://www.codearmo.com/blog/sharpe-sortino-and-calmar-ratios-python)

In [ ]:
df_storico['transaction_date'].min()

In [ ]:
weights = [
    df_pivot[df_pivot['ticker_yf'].eq(x_)]['weight_pf'].values[0]
    for x_ in df.columns
]

In [ ]:
df_weighted = df.copy()
df_weighted['weighted_average'] = np.average(df, weights=weights, axis=1)

In [ ]:
df_weighted

In [ ]:
import datetime

begin_date = df_storico['transaction_date'].min()
today = datetime.datetime.now().date()

date_range = pd.date_range(start=begin_date, end=today, freq='D')

In [ ]:
df_asset_allocation = pd.DataFrame(
    index=date_range,
    columns=ticker_list,
    data=0,
    dtype=int,
)

for (data, ticker), group in df_storico[
    df_storico['ticker_yf'].ne('EGLN.L')
].groupby(['transaction_date', 'ticker_yf']):
    total_shares = group['shares'].sum()
    df_asset_allocation.loc[data, ticker] += total_shares
    
df_asset_allocation = df_asset_allocation.cumsum()

In [ ]:
# Crescita patrimonio
df_wealth = df_asset_allocation.multiply(
    df.loc[begin_date:]
).fillna(method='ffill').sum(axis=1).rename("ap_daily_value")

## Correlation

In [ ]:
def color_df(val: float) -> str:
    if val <= 0.3:
        color = 'darkblue'
    elif (val > 0.3 and val <= 0.7):
        color = 'darkorange'
    elif (val > 0.7 and val < 1.0):
        color = 'darkred'
    elif val == 1.0:
        color = 'white'
    return 'color: %s' % color

In [ ]:
df_corr = df.corr()

df_corr.style.applymap(color_df)

In [ ]:
mask = np.tril(
    np.ones_like(df_corr, dtype=bool)
)

fig = go.Figure(go.Heatmap(
    z=df_corr.mask(mask),
    x=df_corr.columns,
    y=df_corr.columns,
    colorscale=px.colors.diverging.RdBu,
    reversescale=True,
    zmin=-1,
    zmax=1
))

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.show()

## PyPortfolioOpt

[risk-free rate](https://www.ecb.europa.eu/stats/financial_markets_and_interest_rates/euro_short-term_rate/html/index.en.html) area Euro

[Fred](https://fred.stlouisfed.org/series/ECBESTRVOLWGTTRMDMNRT)

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns, plotting

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Risk-free rate
risk_free_rate = 0.0314

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
max_sharpe_weights = ef.max_sharpe(risk_free_rate=risk_free_rate)
ef.portfolio_performance(verbose=True, risk_free_rate=risk_free_rate);

In [ ]:
for it_ in max_sharpe_weights.items():
    print(it_)

In [ ]:
ef_plt = EfficientFrontier(mu, S)

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(
    ef_plt,
    ax=ax,
    show_assets=True,
)
plt.show()

In [ ]:
ef_plt = EfficientFrontier(mu, S)

fig, ax = plt.subplots()
ef_max_sharpe = ef_plt
plotting.plot_efficient_frontier(
    ef_plt,
    ax=ax,
    show_assets=False,
)

# Find the tangency portfolio
ef_max_sharpe.max_sharpe()
ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

# Generate random portfolios
n_samples = 10000
w = np.random.dirichlet(np.ones(ef_plt.n_assets), n_samples)
rets = w.dot(ef_plt.expected_returns)
stds = np.sqrt(np.diag(w @ ef_plt.cov_matrix @ w.T))
sharpes = rets / stds
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

# Output
ax.set_title("Efficient Frontier with random portfolios")
ax.set_xlim((0.0, 1.0))
ax.set_ylim((0.0, 0.2))
ax.legend()
plt.tight_layout()
plt.show()

## Efficient Frontier

In [ ]:
df_returns = df.pct_change()[1:]

In [ ]:
# Annualized returns (cumulative appreciation)
r = (
    (1 + df_returns).prod()
)**(
    252 / df_returns.shape[0]
) - 1

In [ ]:
# Covariance matrix
cov = 252 * df_returns.cov()

In [ ]:
e = np.ones(r.shape[0])

In [ ]:
# Investable universe
icov = np.linalg.inv(cov)

h = np.matmul(e, icov)
g = np.matmul(r, icov)

a = np.sum(e * h)
b = np.sum(r * h)
c = np.sum(r * g)
d = a * c - b**2

In [ ]:
# MVP (minimum-variance portfolio)
mvp = h / a
mvp_return = b / a
mvp_risk = 1 / np.sqrt(a)

In [ ]:
# Tangency portfolio (with zero risk-free rate)
tangency = g / b
tangency_return = c / b
tangency_risk = np.sqrt(c) / b

In [ ]:
mvp_return, mvp_risk